# LangChain: Evaluation

## Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
llm_model = "gpt-4o-mini"

## Create our QandA application

In [3]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain_openai import OpenAIEmbeddings

In [4]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [5]:
embeddings = OpenAIEmbeddings()
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings
).from_loaders([loader])

In [6]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True
)

### Coming up with test datapoints

In [7]:
data[10]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.")

In [8]:
data[11]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 11}, page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.')

### Hard-coded examples

In [9]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

### LLM-Generated examples

In [10]:
from langchain.evaluation.qa.generate_chain import QAGenerateChain


In [11]:
example_gen_chain = QAGenerateChain.from_llm(llm=ChatOpenAI(model=llm_model))

In [12]:
new_examples = example_gen_chain.apply(
    [{"doc": t} for t in data[:5]]
)

In [23]:
modified_new_examples = [data.get("qa_pairs") for data in new_examples]
modified_new_examples

[{'query': "What are the key features of the Women's Campside Oxfords as described in the document, including materials, construction details, and design elements?",
  'answer': "The Women's Campside Oxfords are designed to be ultracomfortable, featuring a super-soft canvas material that provides a broken-in feel from the first wear. They have thick cushioning and quality construction. The shoes have an EVA innersole that includes Cleansport NXT® antimicrobial odor control, a vintage hunt, fish, and camping motif, and a moderate arch contour. Additionally, the EVA foam midsole offers cushioning and support, while the outsole is made of molded rubber with a modified chain-tread pattern, inspired by chain-tread designs. The approximate weight of a pair is 1 lb. 1 oz. The sizing recommendation is to order regular shoe size, or to order up to the next whole size for half sizes that are not offered."},
 {'query': 'What are the dimensions of the small and medium sizes of the Recycled Waterho

In [24]:
data[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

### Combine examples

In [15]:
examples += modified_new_examples
examples

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'query': "What are the key features of the Women's Campside Oxfords as described in the document, including materials, construction details, and design elements?",
  'answer': "The Women's Campside Oxfords are designed to be ultracomfortable, featuring a super-soft canvas material that provides a broken-in feel from the first wear. They have thick cushioning and quality construction. The shoes have an EVA innersole that includes Cleansport NXT® antimicrobial odor control, a vintage hunt, fish, and camping motif, and a moderate arch contour. Additionally, the EVA foam midsole offers cushioning and support, while the outsole is made of molded rubber with a modified chain-tread pattern, inspired by chain-tread designs. The approximate weight of a pair is 1 lb. 1 oz

In [16]:
qa.invoke(examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'result': 'Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.'}

## Manual Evaluation

In [17]:
import langchain
langchain.debug = True

In [18]:
qa.invoke(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-

{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'result': 'Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.'}

## LLM assisted evaluation

In [19]:
predictions = qa.batch(examples)
print(predictions)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "answer": "Yes"
}[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?",
  "answer": "The DownTek collection"
}

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What are the key features of the Women's Campside Oxfords as described in the document, including materials, construction details, and design elements?",
  "answer": "The Women's Campside Oxfords are designed to be ultracomfortable, featuring a super-soft canvas material that provides a broken-in feel from the first wear. They have thick cushioning and quality construction. The shoes have an EVA innersole that includes Cleansport NXT® antimicrobial odor control, a vintage hunt, fish, and camping motif, and a moderate arch contour. Additionally, the EVA fo

In [20]:
langchain.debug = False

from langchain.evaluation.qa import QAEvalChain

In [21]:
llm = ChatOpenAI(temperature=0, model=llm_model)
eval_chain = QAEvalChain.from_llm(llm)

In [26]:
graded_outputs = eval_chain.evaluate(examples, predictions)
graded_outputs

[{'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'GRADE: CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'GRADE: CORRECT'},
 {'results': 'GRADE: CORRECT'},
 {'results': 'GRADE: CORRECT'}]

In [27]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.
Predicted Grade: CORRECT

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.
Predicted Grade: CORRECT

Example 2:
Question: What are the key features of the Women's Campside Oxfords as described in the document, including materials, construction details, and design elements?
Real Answer: The Women's Campside Oxfords are designed to be ultracomfortable, featuring a super-soft canvas material that provides a broken-in feel from the first wear. They have thick cushioning and quality construction. The shoes have an EVA innersole that includes Cleansport NXT® antimicrobial odor control, a vintage hunt, fish, and camping motif

In [27]:
graded_outputs[0]

{'results': 'CORRECT'}